## 📘 Usage Examples

This notebook shows how to use key functions from the `lattice_methods` module.
---

### 🔹 Example 1: Reduce a 2D Basis
### 🔧 Function: `reduce_2d_basis`

Performs iterative 2D lattice basis reduction using projection and subtraction (similar to Gram-Schmidt).
Returns a list of intermediate steps for inspection or visualization.

---

#### 📥 Parameters

| Name      | Type         | Description                                             |
|-----------|--------------|---------------------------------------------------------|
| `basis1`  | `np.ndarray` | First 2D basis vector (shape `(2,)`)                   |
| `basis2`  | `np.ndarray` | Second 2D basis vector (shape `(2,)`)                  |
| `verbose` | `bool`       | If `True`, prints step-by-step details to the console. Default is `False` |

---

#### 📤 Returns

A `List[Dict]` of reduction steps. Each step contains:
- `'step'`: step index (starting from 0)
- `'b1'`: current state of the first basis vector
- `'b2'`: current state of the second basis vector

---




#### 💡 Example

In [2]:
from lattice_methods import reduce_2d_basis
from lattice_methods import lll_reduce
import numpy as np
import pandas as pd
from tabulate import tabulate


# 2d vector example
b1 = np.array([58, 19])
b2 = np.array([168, 55])

data = reduce_2d_basis(b1, b2, verbose=True)

for row in data:
    row["b1"] = str(row["b1"])
    row["b2"] = str(row["b2"])

print(tabulate(data, headers="keys", tablefmt="pipe"))

| step       | b1      | b2        |
|:-----------|:--------|:----------|
| 0          | [58 19] | [168  55] |
| 1          | [-6 -2] | [58 19]   |
| 2          | [-2 -1] | [-6 -2]   |
| 3          | [0 1]   | [-2 -1]   |
| 4          | [0 1]   | [-2  0]   |
| → shortest | [0 1]   |           |


### 🔹 Example 2: Reduce a 2D Basis and higher
### 🔧 Function: `lll_reduce`

Implements the **Lenstra–Lenstra–Lovász (LLL)** lattice basis reduction algorithm for integer bases in arbitrary dimension.
Applies size reduction and swaps based on the Lovász condition to produce shorter, nearly orthogonal vectors.

---

#### 📥 Parameters

| Name       | Type           | Description                                                                 |
|------------|----------------|-----------------------------------------------------------------------------|
| `basis`    | `List[np.ndarray]` | List of linearly independent integer vectors (dimension `n`)                |
| `delta`    | `float`        | Lovász parameter, typically between 0.5 and 1. Default is `0.75`            |
| `verbose`  | `bool`         | If `True`, prints the internal steps of reduction. Default is `False`       |

---

#### 📤 Returns

- `List[np.ndarray]`:
  The reduced basis as a list of vectors in the same dimension as the input.

---

#### 💡 Example Usage

In [3]:
data = lll_reduce([b1,b2], verbose=False)
print(data)

[array([0, 1]), array([-2,  0])]


# 🔐 NTRU Public Key Encryption — Python Implementation

This notebook demonstrates the usage of NTRU-based encryption and decryption using polynomials in modular arithmetic.

**📌 Setup**

We use the following parameters:

- `N ` — degree of the ring (modulo \( x^N - 1 \))
- `p ` — small modulus (used for message space)
- `q ` — large modulus (used for ciphertext and key space)
- Polynomials:
  - `f` — private polynomial (invertible mod p and q)
  - `g` — public polynomial
  - `phi` — random small polynomial (used for each encryption)
  - `m` — message polynomial (with small coefficients mod p)

**⚙️ Functions**

**🔧 `ntru_generate_keys(N, p, q, g, f)`**

Generates keypair:

- **Inputs:**
  - `N` — ring degree
  - `p`, `q` — small/large moduli
  - `g`, `f` — polynomials (`sympy.Poly`) over integers

- **Returns:**
  - `public_key`: tuple `(N, p, q, h)`
  - `private_key`: tuple `(f, Fp)` (with inverses mod `p` and `q`)

---

**✉️ `ntru_encryption(public_key, phi, m)`**

Encrypts message polynomial `m` using random ephemeral key `phi`.

- **Inputs:**
  - `public_key = (N, p, q, h)`
  - `phi` — random small polynomial (`Poly`)
  - `m` — message polynomial (`Poly`, domain=`GF(p)`)

- **Returns:**
  - `ciphertext` — encrypted `Poly` over `GF(q)`

 🔓 `ntru_decryption(public_key, private_key, ciphertext)`


Decrypts the ciphertext back to message.

- **Inputs:**
  - `public_key = (N, p, q, h)`
  - `private_key = (f, Fp, Fq)`
  - `ciphertext` — polynomial to decrypt

- **Returns:**
  - `message` — decrypted polynomial over `GF(p)`

---

**🧪 Example Usage**
📌 _Numerical values in this example are taken directly from:_ [https://shrek.unideb.hu/~tengely/crypto/section-8.html](https://shrek.unideb.hu/~tengely/crypto/section-8.html)

---


In [4]:
from lattice_methods import ntru_generate_keys, ntru_encryption, ntru_decryption
from sympy import Poly, GF,symbols

x = symbols('x')

N = 7
p = 3
q = 41

#### a_0 x^n + a_1 x^n-1.....
phi = [1, -1, 0, 0, 0, 1, -1]
m = [0, -1, 0, 1, 1, -1, 1]
g = [1, 0, 1, 0, -1, -1, 0]
f = [1, 0, -1, 1, 1, 0, -1]


poly_f = Poly(f, x)
poly_g = Poly(g, x)
poly_m = Poly(m, x)
poly_phi = Poly(phi, x)

pub_key, prv_key = ntru_generate_keys(N, p, q, poly_g, poly_f)
ciphertext = ntru_encryption(pub_key, poly_phi, poly_m)
poly_d = ntru_decryption(pub_key, prv_key, ciphertext)

[f, Fp] = prv_key
[N,p, q, h] = pub_key

poly_m = Poly(poly_m, x, domain=GF(p))
print("🔑 NTRU Key Summary")
print("=" * 80)
print(f"📤 Public Key (N={N}, p={p}, q={q}, h={h}):")
print(f"    h(x) = {h}\n")

print("🔐 Private Key:")
print(f"    f(x)  = {f}")
print(f"    Fp(x) = {Fp}")


print("✉️ Original Message Polynomial:")
print(f"    m(x) = {poly_m}\n")

print("📬 Decrypted Message Polynomial:")
print(f"    m'(x) = {poly_d}")
print("=" * 80)

🔑 NTRU Key Summary
📤 Public Key (N=7, p=3, q=41, h=[19, 38, 6, 32, 24, 37, 8]):
    h(x) = [19, 38, 6, 32, 24, 37, 8]

🔐 Private Key:
    f(x)  = Poly(x**6 - x**4 + x**3 + x**2 - 1, x, domain='ZZ')
    Fp(x) = [1, -1, 0, 1, 1, 1, 1]
✉️ Original Message Polynomial:
    m(x) = Poly(-x**5 + x**3 + x**2 - x + 1, x, modulus=3)

📬 Decrypted Message Polynomial:
    m'(x) = Poly(-x**5 + x**3 + x**2 - x + 1, x, modulus=3)
